In [ ]:
import random
import os
import shutil
import copy
from skimage import measure
from skimage import morphology as skim
import skimage.io as skio
import warnings
import pandas as pd
import numpy as np
import pathlib
import geopandas as gpd
from rasterio import features, coords
from rasterio.plot import reshape_as_raster
import rasterio
from shapely.geometry import shape
from osgeo import gdal
from itertools import product
from rasterio import windows
from multiprocessing.pool import ThreadPool
from cropmask.label_prep import rio_bbox_to_polygon
from cropmask.misc import parse_yaml, make_dirs
from cropmask.grid import *

In [ ]:
import cropmask.preprocess as pp
from cropmask.misc import make_dirs, remove_dirs
wflow = pp.PreprocessWorkflow("/home/ryan/work/CropMask_RCNN/cropmask/preprocess_config.yaml", 
                                 "/mnt/point/western_nebraska_landsat_scenes/LT050320312005082801T1-SC20190418222350/",
                                 "/mnt/point/external/nebraska_pivots_projected.geojson")

In [ ]:
%load_ext autoreload
%autoreload 2
from cropmask import grid

In [ ]:
in_path, out_dir = '/mnt/point/landsat-1024-cp/stacked/LT05_L1TP_032031_20050828_20160912_01_T1_sr.tif', '/mnt/point/landsat-1024-cp/landsat_grid_imgs'
grid_size = 512
output_name_template='tile_{}-{}.tif'


In [ ]:
with rasterio.open(in_path, shared=False) as src:
    chip_list = get_tiles_for_threaded_map(src, width=grid_size, height=grid_size)
chunk_list = chunk_chips(chip_list, 4) # all possible chips, gets chunked into list of lists of chips
out_paths = [] 
chip_list=chunk_list[0]

In [ ]:
chip_list

In [ ]:
outpaths = []
with rasterio.open(in_path, shared=False) as ds:

    for chip in chip_list:
        window, transform = chip
        ds.meta['transform'] = transform
        ds.meta['width'], ds.meta['height'] = window.width, window.height
        outpath = os.path.join(out_dir,output_name_template.format(int(window.col_off), int(window.row_off)))
        with rasterio.open(outpath, 'w', **ds.meta, shared=False) as outds:
            outds.write(ds.read(window=window))
        outpaths.append(outpath)
print(outpaths)

In [ ]:
write_by_window(in_path, out_dir, output_name_template='tile_{}-{}.tif',chip_list=chunk_list[0] )

In [ ]:
out_paths.extend(list(map_threads(lambda x: write_by_window(in_path, out_dir, output_name_template, x), chunk_list))) 

In [ ]:
%%time
grid_images_rasterio_controlled_threads('/mnt/point/landsat-1024-cp/stacked/LT05_L1TP_032031_20050828_20160912_01_T1_sr.tif', '/mnt/point/landsat-1024-cp/landsat_grid_imgs', MAX_THREADS = 100)